In [ ]:
#!pip3 install kfp

In [ ]:
#To build from local (connectwith gcloud cli first)
#kubectl config set-context $(kubectl config current-context) --namespace=kubeflow-rosario-laface

In [123]:
import sys
sys.path.insert(1, '/home/jovyan/.local/lib/python3.6/site-packages')
#See https://www.kubeflow.org/docs/pipelines/sdk/component-development/
import kfp
from kfp import compiler
import kfp.components as comp
import kfp.dsl as dsl
from kfp import gcp

EXPERIMENT_NAME = 'kubeflow-test'
BUCKET = "ai-vqc-kubeflow-components"
OUTPUT_BUCKET='ai-vqc-kubeflow-output'
PROJECT='ai-vqc'

In [124]:
#!gcloud source repos clone kubeflow-test --project=ai-vqc
#%cd kubeflow-test
!git checkout develop
!git pull
#!git add . && git commit -m "" && git push

M	.ipynb_checkpoints/pipeline_creation-checkpoint.ipynb
D	deploy/.ipynb_checkpoints/manual_build-checkpoint.ipynb
M	pipeline_creation.ipynb
Already on 'develop'
Your branch is up to date with 'origin/develop'.
remote: Finding sources: 100% (5/5)
remote: Total 5 (delta 4), reused 5 (delta 4)
Unpacking objects: 100% (5/5), done.
From https://source.developers.google.com/p/ai-vqc/r/kubeflow-test
   c612c4a..c21af4d  develop    -> origin/develop
Updating c612c4a..c21af4d
Fast-forward
 train/src/train.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)


In [125]:
#Docker build from SDK (see: https://www.kubeflow.org/docs/pipelines/sdk/sdk-overview/)
#kfp.containers.build_image_from_working_dir(image_name='eu.gcr.io/ai-vqc/kubeflow_train:test', working_dir='/home/jovyan/kubeflow-test/train')
#kfp.containers.build_image_from_working_dir(image_name='eu.gcr.io/ai-vqc/kubeflow_deploy:test', working_dir='/home/jovyan/kubeflow-test/deploy')

In [126]:
#Copy component definitions to GCS
!./copy_specifications.sh $BUCKET $PROJECT

#Don't forget to rebuild updated components with <component_folder>/build_image.sh

#Preprocess
#preprocess_operation = kfp.components.load_component_from_url('https://storage.googleapis.com/{}/components/preprocess/component.yaml'.format(BUCKET))
#help(preprocess_operation)

#Train
train_operation = kfp.components.load_component_from_url('https://storage.googleapis.com/{}/components/train/component.yaml'.format(BUCKET))
help(train_operation)

#Deploy
ai_platform_deploy_operation = comp.load_component_from_url("https://storage.googleapis.com/{}/components/deploy/component.yaml".format(BUCKET))
help(ai_platform_deploy_operation)

Creating gs://ai-vqc-kubeflow-components/...
ServiceException: 409 Bucket ai-vqc-kubeflow-components already exists.

Copy component specifications to Google Cloud Storage
Removing gs://ai-vqc-kubeflow-components/components/train/component.yaml...
/ [1 objects]                                                                   
Operation completed over 1 objects.                                              
Copying file://train/component.yaml [Content-Type=application/octet-stream]...
/ [1 files][  603.0 B/  603.0 B]                                                
Operation completed over 1 objects/603.0 B.                                      
Updated ACL on gs://ai-vqc-kubeflow-components/components/train/component.yaml
Removing gs://ai-vqc-kubeflow-components/components/deploy/component.yaml...
/ [1 objects]                                                                   
Operation completed over 1 objects.                                              
Copying file://deploy/compon

In [127]:
#Build the pipeline components 
@dsl.pipeline(
  name='Kubeflow Test Pipeline',
  description='Performs preprocessing, training and deployment.'
)
def pipeline():
    # Input name "Input 1" is converted to pythonic parameter name "input_1"
    
    #Preprocess
    '''
    preprocess_task = preprocess_operation(
        input_1_uri='gs://kubeflow-examples-data/named_entity_recognition_dataset/ner.csv,
        output_y_uri_template="gs://{}/{{workflow.uid}}/preprocess/y/data".format(BUCKET),
        output_x_uri_template="gs://{}/{{workflow.u        output_model_uri='/opt/ml/output/',
id}}/preprocess/x/data".format(BUCKET),
        output_preprocessing_state_uri_template="gs://{}/{{workflow.uid}}/model".format(BUCKET)
    ).apply(kfp.gcp.use_gcp_secret('user-gcp-sa')) 
    '''
    
    #Train
    train_task = train_operation(
        input_1='dpc-covid19-ita-regioni.csv',
        output_model="gs://{}/model".format(OUTPUT_BUCKET),
    ).apply(kfp.gcp.use_gcp_secret('user-gcp-sa'))
    
    
    #Deploy
    deploy_task = ai_platform_deploy_operation(
        model_path=train_task.output,
        model_name="kubeflow_test",
        model_region="europe-west1",
        model_version="version5",
        model_runtime_version="1.14",
        model_prediction_class="model_prediction.CustomModelPrediction",
        model_python_version="3.5",
        model_package_uris="gs://{}/routine/custom_prediction_routine-0.2.tar.gz".format(BUCKET)
    ).after(train_task).apply(kfp.gcp.use_gcp_secret('user-gcp-sa'))
    

In [128]:
#Compile the pipeline

pipeline_func = pipeline
pipeline_filename = pipeline_func.__name__ + '.pipeline.zip'

import kfp.compiler as compiler
compiler.Compiler().compile(pipeline_func, pipeline_filename) #compiles your Python domain-specific language (DSL) code into a single static configuration (in YAML format) that the Kubeflow Pipelines service can process.

In [129]:
#Create Kubeflow experiment

client = kfp.Client()

try:
    experiment = client.get_experiment(experiment_name=EXPERIMENT_NAME)
except:
    experiment = client.create_experiment(EXPERIMENT_NAME)
    
print(experiment)

{'created_at': datetime.datetime(2020, 3, 11, 12, 45, 17, tzinfo=tzlocal()),
 'description': None,
 'id': '319ba217-4d69-40f2-8708-fcf58fd4008d',
 'name': 'kubeflow-test'}


In [130]:
#Run the pipeline

arguments = {}

run_name = pipeline_func.__name__ + ' run'
run_result = client.run_pipeline(experiment.id, 
                                 run_name, 
                                 pipeline_filename, 
                                 arguments)

print(experiment.id)
print(run_name)
print(pipeline_filename)
print(arguments)

319ba217-4d69-40f2-8708-fcf58fd4008d
pipeline run
pipeline.pipeline.zip
{}
